# Test multi instance and multi user session

In [ ]:
!lamin login testuser1
!lamin init --storage "./testsetup-prepare"

In [ ]:
import lamindb_setup as ln_setup
import pytest

In [ ]:
ln_setup.core.django.reset_django()  # django is not configured, so no effect

In [ ]:
assert ln_setup.settings.user.handle == "testuser1"

In [ ]:
ln_setup.init(storage="./testsetup")

In [ ]:
from lamindb.models import User

In [ ]:
User.objects.get(handle="testuser1")

In [ ]:
with pytest.raises(Exception):  # does not exist
    User.objects.get(handle="testuser2")

Let us login with another user:

In [ ]:
ln_setup.login("testuser2")

In [ ]:
User.objects.get(handle="testuser2")

Let us try to init another instance in the same Python session:

In [ ]:
from lamindb_setup._init_instance import CannotSwitchDefaultInstance

with pytest.raises(CannotSwitchDefaultInstance):
    ln_setup.init(storage="./testsetup2")
with pytest.raises(CannotSwitchDefaultInstance):
    ln_setup.connect("testsetup")
with pytest.raises(RuntimeError):
    ln_setup.migrate.create()
with pytest.raises(RuntimeError):
    ln_setup.migrate.deploy()

assert ln_setup.settings.instance.slug == "testuser1/testsetup"

Reset `django` and connect to another instance:

In [ ]:
ln_setup.core.django.reset_django()

In [ ]:
ln_setup.connect("testuser1/testsetup-prepare")

In [ ]:
assert ln_setup.settings.instance.slug == "testuser1/testsetup-prepare"

In [ ]:
!lamin login testuser1
!lamin delete --force testsetup-prepare
!lamin delete --force testsetup